In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Hyperfine solutions for H2S molecule (spin-rotation interaction)



In [4]:
import h5py
import numpy as np

from rovib.spinrot_xy2 import spinrot_xy2
from rovib.cartens import CART_IND
from rovib.symtop import threej_wang

Read vibrational matrix elements of spin-rotation tensors and compute their matrix elements in rovibrational basis

In [5]:
def rovib_states(j: int, pmax: int = 20):
    """Reads rovibrational energies and quanta from files for different values
    of the J quantum number (see `h2s_rovib.ipynb`).
    """
    h5 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j}.h5", "r")
    energies = {}
    quanta = {}
    for sym in h5["energies"].keys():
        enr = h5["energies"][sym][:]
        coefs = h5["coefficients"][sym][:]
        vind = h5["vib-indices"][sym][:]
        rind = h5["rot-indices"][sym][:]
        qua = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5["quanta"][sym][:]]
        )
        energies[sym] = enr
        quanta[sym] = qua
    return energies, quanta

In [6]:
def tensor_rovib_me(
    rank: int, j1: int, j2: int, vib_me, pmax: int = 20, linear: bool = False, tol=1e-12
):
    """Computes rovibrational matrix elements of a Cartesian tensor operator.

    This function calculates the rovibrational matrix elements using wavefunctions stored in
    separate files for different values of the J quantum number (see `h2s_rovib.ipynb`).
    The computation is performed for the specified bra (`j1`) and ket (`j2`) values of the J
    quantum number.
    """

    # determine the order of Cartesian indices in the Cartesian-to-spherical tensor
    #   transformation matrix (in cartens.CART_IND and symtop.threej_wang)
    cart_ind = [["xyz".index(x) for x in elem] for elem in CART_IND[rank]]

    # reshape vibrational matrix elements such that the order of Cartesian indices
    #   correspond to the order in symtop.threej_wang output
    vib_me_ = np.moveaxis([vib_me[:, :, i, j] for (i, j) in cart_ind], 0, -1)

    # compute rotational matrix elements of three-j symbol contracted with
    #   Cartesian-to-spherical tensor transformation matrix
    jktau_list1, jktau_list2, rot_me = threej_wang(rank, j1, j2, linear=linear)
    # rot_me[omega].shape = (2*j1+1, 2*j2+1, ncart)

    h5_1 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j1}.h5", "r")
    h5_2 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j2}.h5", "r")

    res = {}

    for sym1 in h5_1["energies"].keys():
        enr1 = h5_1["energies"][sym1][:]
        coefs1 = h5_1["coefficients"][sym1][:]
        vind1 = h5_1["vib-indices"][sym1][:]
        rind1 = h5_1["rot-indices"][sym1][:]
        qua1 = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5_1["quanta"][sym1][:]]
        )

        for sym2 in h5_2["energies"].keys():
            enr2 = h5_2["energies"][sym2][:]
            coefs2 = h5_2["coefficients"][sym2][:]
            vind2 = h5_2["vib-indices"][sym2][:]
            rind2 = h5_2["rot-indices"][sym2][:]
            qua2 = np.array(
                [elem[0].decode("utf-8").split(",") for elem in h5_2["quanta"][sym2][:]]
            )

            me = []
            for omega in range(rank + 1):
                me_ = np.einsum(
                    "ijc,ijc->ij",
                    vib_me_[np.ix_(vind1, vind2)],
                    rot_me[omega][np.ix_(rind1, rind2)],
                    optimize="optimal",
                )
                me.append(
                    np.einsum(
                        "ik,ij,jl->kl", np.conj(coefs1), me_, coefs2, optimize="optimal"
                    )
                )

            me = np.moveaxis(np.array(me), 0, -1)
            me[np.abs(me) < tol] = 0
            if np.count_nonzero(me) > 0:
                res[(sym1, sym2)] = np.array(me)

    return res

In [7]:
pmax = 20

# read vibrational matrix elements of spin-rotation tensors

with h5py.File(f"h2s_vibme_pmax{pmax}.h5", "r") as h5:
    sr_h1_vib = h5["spin-rotation"]["h1"][:]
    sr_h2_vib = h5["spin-rotation"]["h2"][:]

# compute rovibrational matrix elements for different pairs of J quanta

rank = 2
min_j = 0
max_j = 4

rovib_enr = {}
rovib_qua = {}
for j in range(min_j, max_j + 1):
    rovib_enr[j], rovib_qua[j] = rovib_states(j)

rovib_sr1_me = {}
rovib_sr2_me = {}
me_sr1 = {}
for j1 in range(min_j, max_j + 1):
    for j2 in range(min_j, max_j + 1):
        rovib_sr1_me[(j1, j2)] = tensor_rovib_me(rank, j1, j2, sr_h1_vib)
        rovib_sr2_me[(j1, j2)] = tensor_rovib_me(rank, j1, j2, sr_h2_vib)

In [19]:
for sym, val in rovib_sr1_me[(0, 2)].items():
    print(sym)
    print(val[0,0,:])
    print(val[0,1,:])
    print(val[0,2,:])
    print(val[0,3,:])
    print(val[0,4,:])


('A1', 'A1')
[ 0.        +0.j  0.        +0.j -1.21239949+0.j]
[ 0.        +0.j  0.        +0.j -0.63432652+0.j]
[0.        +0.j 0.        +0.j 0.00158073+0.j]
[0.        +0.j 0.        +0.j 0.00528235+0.j]
[0.        +0.j 0.        +0.j 0.00041785+0.j]
('A1', 'B2')
[ 0.        +0.j  0.        +0.j 12.20517419+0.j]
[ 0.        +0.j  0.        +0.j -0.00336005+0.j]
[0.        +0.j 0.        +0.j 0.00414036+0.j]
[ 0.        +0.j  0.        +0.j -0.01483761+0.j]
[0.        +0.j 0.        +0.j 0.03046567+0.j]
('B2', 'A1')
[0.        +0.j 0.        +0.j 0.03321834+0.j]
[ 0.        +0.j  0.        +0.j -0.00925734+0.j]
[ 0.        +0.j  0.        +0.j -0.10239023+0.j]
[ 0.        +0.j  0.        +0.j -0.01314003+0.j]
[0.        +0.j 0.        +0.j 0.02775036+0.j]
('B2', 'B2')
[0.        +0.j 0.        +0.j 0.00385182+0.j]
[ 0.        +0.j  0.        +0.j -0.01078235+0.j]
[ 0.        +0.j  0.        +0.j -0.00294702+0.j]
[0.        +0.j 0.        +0.j 0.00578717+0.j]
[ 0.        +0.j  0.     

In [8]:
f = 1

enr, vec, qua = spinrot_xy2(
    f,
    rovib_enr,
    rovib_qua,
    rovib_sr1_me,
    rovib_sr2_me,
    spin_states=((0, "B2"), (1, "A1")),
    spins=(0.5, 0.5)
)
for key, val in enr.items():
    print(key, val.shape, val)

B1 (1517,) [ 3304.90655447  3306.25036378  3346.32166991 ... 28095.81400952
 28152.60879763 38855.620538  ]
B2 (1452,) [ 3310.53577666  3329.45817342  4493.67391267 ... 28124.183249
 38857.87783848 38876.34592702]


In [9]:
zpe = np.min([np.min(elem_) for elem in rovib_enr.values() for elem_ in elem.values()])
print("zpe:", zpe)

# print(rovib_enr[1]['B1']-zpe)
print(enr['B1']-zpe)


zpe: 3291.1602143624377
[1.37463401e+01 1.50901494e+01 5.51614555e+01 ... 2.48046538e+04
 2.48614486e+04 3.55644603e+04]


In [12]:
for sym, q in qua.items():
    print(sym, q)

B1 [(1, 'A2', 0, 'B2'), (1, 'B1', 1, 'A1'), (2, 'B1', 1, 'A1')]
B2 [(0, 'B2', 1, 'A1'), (1, 'A1', 0, 'B2'), (1, 'B2', 1, 'A1'), (2, 'B2', 1, 'A1')]
